# Инициализация

Загружаем библиотеки необходимые для выполнения кода ноутбука.

### Imports

In [1]:
import os
import requests
from dotenv import load_dotenv

import pandas as pd
import polars as pl
import numpy as np
import scipy

import matplotlib.pyplot as plt
import seaborn as sns


from implicit.als import AlternatingLeastSquares
from sklearn.preprocessing import LabelEncoder

from catboost import CatBoostClassifier, Pool

import calendar
import gc
import joblib

/home/mle-user/mle_projects/mle-project-sprint-4/.venv_recsys/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Config

In [2]:
RANDOM_STATE = 42

# Load environment variables
load_dotenv()

# Dataset URLs from .env
datasets = {
    'tracks.parquet': os.getenv('RAW_URL_TRACKS'),
    'catalog_names.parquet': os.getenv('RAW_URL_CATALOG_NAMES'),
    'interactions.parquet': os.getenv('RAW_URL_INTERACTIONS'),
}
raw_dir = os.getenv('RAW_DATA_DIR', '../data/raw')

# === ЭТАП 1 ===

# Загрузка первичных данных

Загружаем первичные данные из файлов:
- tracks.parquet
- catalog_names.parquet
- interactions.parquet

In [3]:
# ---------- Download datasets and save locally ---------- #
# Create directory
os.makedirs(raw_dir, exist_ok=True)

# Download and save each dataset
for filename, url in datasets.items():
    save_path = os.path.join(raw_dir, filename)
    response = requests.get(url)
    response.raise_for_status()
    with open(save_path, 'wb') as f:
        f.write(response.content)
    print(f'Saved {filename} to {save_path}')

Saved tracks.parquet to ../data/raw/tracks.parquet
Saved catalog_names.parquet to ../data/raw/catalog_names.parquet
Saved interactions.parquet to ../data/raw/interactions.parquet


In [4]:
# ---------- Load datasets ---------- #
tracks = pl.read_parquet(os.path.join(raw_dir, 'tracks.parquet'))
catalog_names = pl.read_parquet(os.path.join(raw_dir, 'catalog_names.parquet'))
interactions = pl.read_parquet(os.path.join(raw_dir, 'interactions.parquet'))

# Обзор данных

Проверяем данные, есть ли с ними явные проблемы.

In [5]:
# ---------- Check data summary ---------- #
def data_summary(df: pd.DataFrame, name: str):
    '''
        Display a quick overview of a DataFrame.
    '''

    print(f'\n===== {name.upper()} =====')  
  
    # Sample rows
    print('\nSample rows:')
    display(df.head())

    # Shape
    rows, cols = df.shape
    print(f'\nShape: {rows} rows x {cols} columns')
    
    # Data info
    print('\nSummary for numeric columns:')
    print(df.describe())
   
    # Unique values (column-wise, skip if error occurs)
    print('\nUnique values (for each column):')
    try:
        for col in df.columns:
            print(f'\nColumn: {col}')
            print(df[col].value_counts())
    except Exception as e:
        print(f'Skipped value_counts due to error: {e}')
    
    # Missing values
    print('\nMissing values:')
    print(df.null_count())

In [6]:
data_summary(tracks, 'tracks')


===== TRACKS =====

Sample rows:


track_id,albums,artists,genres
i64,list[i64],list[i64],list[i64]
26,"[3, 2490753]",[16],"[11, 21]"
38,"[3, 2490753]",[16],"[11, 21]"
135,"[12, 214, 2490809]",[84],[11]
136,"[12, 214, 2490809]",[84],[11]
138,"[12, 214, … 17294156]",[84],[11]



Shape: 1000000 rows x 4 columns

Summary for numeric columns:
shape: (9, 5)
┌────────────┬──────────────┬────────┬─────────┬────────┐
│ statistic  ┆ track_id     ┆ albums ┆ artists ┆ genres │
│ ---        ┆ ---          ┆ ---    ┆ ---     ┆ ---    │
│ str        ┆ f64          ┆ f64    ┆ f64     ┆ f64    │
╞════════════╪══════════════╪════════╪═════════╪════════╡
│ count      ┆ 1e6          ┆ 1e6    ┆ 1e6     ┆ 1e6    │
│ null_count ┆ 0.0          ┆ 0.0    ┆ 0.0     ┆ 0.0    │
│ mean       ┆ 3.6851e7     ┆ null   ┆ null    ┆ null   │
│ std        ┆ 2.6798e7     ┆ null   ┆ null    ┆ null   │
│ min        ┆ 26.0         ┆ null   ┆ null    ┆ null   │
│ 25%        ┆ 1.5430883e7  ┆ null   ┆ null    ┆ null   │
│ 50%        ┆ 3.4550746e7  ┆ null   ┆ null    ┆ null   │
│ 75%        ┆ 5.6925472e7  ┆ null   ┆ null    ┆ null   │
│ max        ┆ 1.01521819e8 ┆ null   ┆ null    ┆ null   │
└────────────┴──────────────┴────────┴─────────┴────────┘

Unique values (for each column):

Column: track_id
s

In [7]:
data_summary(catalog_names, 'catalog_names')


===== CATALOG_NAMES =====

Sample rows:


id,type,name
i64,str,str
3,"""album""","""Taller Children"""
12,"""album""","""Wild Young Hearts"""
13,"""album""","""Lonesome Crow"""
17,"""album""","""Graffiti Soul"""
26,"""album""","""Blues Six Pack"""



Shape: 1812471 rows x 3 columns

Summary for numeric columns:
shape: (9, 4)
┌────────────┬──────────────┬─────────┬─────────────────────────────────┐
│ statistic  ┆ id           ┆ type    ┆ name                            │
│ ---        ┆ ---          ┆ ---     ┆ ---                             │
│ str        ┆ f64          ┆ str     ┆ str                             │
╞════════════╪══════════════╪═════════╪═════════════════════════════════╡
│ count      ┆ 1.812471e6   ┆ 1812471 ┆ 1812471                         │
│ null_count ┆ 0.0          ┆ 0       ┆ 0                               │
│ mean       ┆ 2.3216e7     ┆ null    ┆ null                            │
│ std        ┆ 2.5260e7     ┆ null    ┆ null                            │
│ min        ┆ 0.0          ┆ album   ┆ 	Drake                          │
│ 25%        ┆ 3.480527e6   ┆ null    ┆ null                            │
│ 50%        ┆ 1.211436e7   ┆ null    ┆ null                            │
│ 75%        ┆ 3.773825e7   ┆ null 

In [8]:
data_summary(interactions, 'interactions')


===== INTERACTIONS =====

Sample rows:


user_id,track_id,track_seq,started_at,__index_level_0__
i32,i32,i16,datetime[ns],i64
0,99262,1,2022-07-17 00:00:00,0
0,589498,2,2022-07-19 00:00:00,1
0,590262,3,2022-07-21 00:00:00,2
0,590303,4,2022-07-22 00:00:00,3
0,590692,5,2022-07-22 00:00:00,4



Shape: 222629898 rows x 5 columns

Summary for numeric columns:
shape: (9, 6)
┌────────────┬───────────────┬──────────────┬──────────────┬───────────────────┬───────────────────┐
│ statistic  ┆ user_id       ┆ track_id     ┆ track_seq    ┆ started_at        ┆ __index_level_0__ │
│ ---        ┆ ---           ┆ ---          ┆ ---          ┆ ---               ┆ ---               │
│ str        ┆ f64           ┆ f64          ┆ f64          ┆ str               ┆ f64               │
╞════════════╪═══════════════╪══════════════╪══════════════╪═══════════════════╪═══════════════════╡
│ count      ┆ 2.22629898e8  ┆ 2.22629898e8 ┆ 2.22629898e8 ┆ 222629898         ┆ 2.22629898e8      │
│ null_count ┆ 0.0           ┆ 0.0          ┆ 0.0          ┆ 0                 ┆ 0.0               │
│ mean       ┆ 687576.660624 ┆ 3.6536e7     ┆ 462.140293   ┆ 2022-08-29        ┆ 461.140293        │
│            ┆               ┆              ┆              ┆ 16:39:44.541353   ┆                   │
│ std       

### Main takeaways
1. Tracs dataframe contains lists instead of scalar values. This can cause several problems:
- Unable to get insights on data;
Counting, merging, or joining on list columns is tricky.
- Missing values are hidden;
df.isna().sum() can’t detect empty lists, so there might be tracks with no genres or artists but they look as not missing values.
- Hard to work with for ML models.
Most algorithms expect scalar values, not lists.
Thus, it's necessary to explode the lists to get a dataframe with one row per track-per-item (artist, genre, album).

2. Catalog_names dataframe is in a format where everything (tracks, albums, artists, genres) is stacked in one column, and the type column tells what each row represents. This format isn't convenient for futher work. Thus, it's necessary to split catalog_names into several dataframes.

# Data Preprocessing

In [9]:
# ---------- Explode lists of tracs dataframe into separate rows ---------- #
tracks_exploded = (
    tracks
        .explode('albums')
        .explode('artists')
        .explode('genres')
        .rename({
            'albums': 'album_id',
            'artists': 'artist_id',
            'genres': 'genre_id'
        })
)
data_summary(tracks_exploded, 'tracks_exploded')


===== TRACKS_EXPLODED =====

Sample rows:


track_id,album_id,artist_id,genre_id
i64,i64,i64,i64
26,3,16,11
26,3,16,21
26,2490753,16,11
26,2490753,16,21
38,3,16,11



Shape: 6765311 rows x 4 columns

Summary for numeric columns:
shape: (9, 5)
┌────────────┬──────────────┬─────────────┬─────────────┬────────────┐
│ statistic  ┆ track_id     ┆ album_id    ┆ artist_id   ┆ genre_id   │
│ ---        ┆ ---          ┆ ---         ┆ ---         ┆ ---        │
│ str        ┆ f64          ┆ f64         ┆ f64         ┆ f64        │
╞════════════╪══════════════╪═════════════╪═════════════╪════════════╡
│ count      ┆ 6.765311e6   ┆ 6.765293e6  ┆ 6.723522e6  ┆ 6.728636e6 │
│ null_count ┆ 0.0          ┆ 18.0        ┆ 41789.0     ┆ 36675.0    │
│ mean       ┆ 2.6519e7     ┆ 6.1523e6    ┆ 1.7323e6    ┆ 79.621245  │
│ std        ┆ 2.3055e7     ┆ 5.4661e6    ┆ 2.2447e6    ┆ 116.178962 │
│ min        ┆ 26.0         ┆ 3.0         ┆ 4.0         ┆ 1.0        │
│ 25%        ┆ 5.218471e6   ┆ 2.538306e6  ┆ 41152.0     ┆ 19.0       │
│ 50%        ┆ 2.3550528e7  ┆ 3.607589e6  ┆ 588833.0    ┆ 49.0       │
│ 75%        ┆ 3.7743676e7  ┆ 9.774915e6  ┆ 3.210009e6  ┆ 75.0       │


In [10]:
# ---------- Function to standardize text data ---------- #
def clean_text(colname: str, new_name: str):
    '''
        Performs transformations on text data while preserving non-English text
    '''
    return (
        pl.col(colname)                                 # Column to clean
        .str.normalize('NFKD')                          # normalize Unicode (NFKD)
        .str.replace_all(r'[\p{M}]', '')                # remove combining marks (accents)
        .str.replace_all(r'[^\w\s]', ' ')               # remove punctuation, keep letters/numbers (all languages)
        .str.replace_all(r'\s+', ' ')                   # collapse multiple spaces
        .str.to_lowercase()                             # convert to lowercase
        .str.replace_all(r'^(the |a |an )', '')         # remove English articles
        .str.to_titlecase()                             # convert to title case
        .str.strip_chars()                              # remove leading/trailing spaces
        .alias(new_name)                                # assign new column name
    )


# ---------- Wrapper function to clean and filter ---------- #
def clean_and_filter_catalog(df: pl.DataFrame, col_name: str) -> pl.DataFrame:
    '''
        Apply text cleaning and remove rows that become empty
    '''
    clean_col_name = f'{col_name}_clean'
    return (
        df
        .with_columns([clean_text(col_name, clean_col_name)])
        .filter(pl.col(clean_col_name) != '')  # Drop empty after cleaning
    )

In [11]:
# ---------- Dataframe tracks_catalog set up ---------- #

tracks_catalog = (
    catalog_names
        .filter(pl.col('type') == 'track')
        .select([
            pl.col('id').alias('track_id'),
            pl.col('name').alias('track')
        ])
)

data_summary(tracks_catalog, 'tracks_catalog')


===== TRACKS_CATALOG =====

Sample rows:


track_id,track
i64,str
26,"""Complimentary Me"""
38,"""Momma's Boy"""
135,"""Atticus"""
136,"""24 Hours"""
138,"""Don't Upset The Rhythm (Go Bab…"



Shape: 1000000 rows x 2 columns

Summary for numeric columns:
shape: (9, 3)
┌────────────┬──────────────┬─────────────────────────────────┐
│ statistic  ┆ track_id     ┆ track                           │
│ ---        ┆ ---          ┆ ---                             │
│ str        ┆ f64          ┆ str                             │
╞════════════╪══════════════╪═════════════════════════════════╡
│ count      ┆ 1e6          ┆ 1000000                         │
│ null_count ┆ 0.0          ┆ 0                               │
│ mean       ┆ 3.6851e7     ┆ null                            │
│ std        ┆ 2.6798e7     ┆ null                            │
│ min        ┆ 26.0         ┆ !                               │
│ 25%        ┆ 1.5430883e7  ┆ null                            │
│ 50%        ┆ 3.4550746e7  ┆ null                            │
│ 75%        ┆ 5.6925472e7  ┆ null                            │
│ max        ┆ 1.01521819e8 ┆ 🧺 Picnics (S04E07) +           │
│            ┆              

In [12]:
# ---------- Clean tracks_catalog from hidden duplicates ---------- #

tracks_catalog = clean_and_filter_catalog(tracks_catalog, 'track')

track_duplicates = (
    tracks_catalog
        .group_by('track_clean')
        .agg([pl.len().alias('count')])
        .filter(pl.col('count') > 1)
        .sort('count')
)

print('Duplicated tracks found:')
print(track_duplicates)

tracks_dedup = (
    tracks_catalog
        .group_by('track_clean')
        .agg(pl.min('track_id').alias('track_id_canonical'))
)

track_id_map = (
    tracks_catalog
        .join(tracks_dedup, on='track_clean', how='left')
        .select(['track_id', 'track_id_canonical'])
)

Duplicated tracks found:
shape: (124_128, 2)
┌────────────────────────────────┬───────┐
│ track_clean                    ┆ count │
│ ---                            ┆ ---   │
│ str                            ┆ u32   │
╞════════════════════════════════╪═══════╡
│ Little Bitter                  ┆ 2     │
│ Нет Имени                      ┆ 2     │
│ Песня Американских Подводников ┆ 2     │
│ Совпали                        ┆ 2     │
│ 1 000 000 Times                ┆ 2     │
│ …                              ┆ …     │
│ Time                           ┆ 302   │
│ Весна                          ┆ 312   │
│ Home                           ┆ 358   │
│ Мама                           ┆ 512   │
│ Intro                          ┆ 525   │
└────────────────────────────────┴───────┘


In [13]:
# ---------- Clean Tracks Catalog ----------
tracks_catalog_clean = (
    tracks_dedup
    .join(tracks_catalog.select(['track_clean', 'track']), on='track_clean', how='left')
    .select([
        pl.col('track_id_canonical').alias('track_id'),
        pl.col('track_clean').alias('track')
    ])
    .unique()
)

In [14]:
# ---------- Dataframe albums_catalog set up ---------- #

albums_catalog = (
    catalog_names
        .filter(pl.col('type') == 'album')
        .select([
            pl.col('id').alias('album_id'),
            pl.col('name').alias('album')
        ])
)

data_summary(albums_catalog, 'albums_catalog')


===== ALBUMS_CATALOG =====

Sample rows:


album_id,album
i64,str
3,"""Taller Children"""
12,"""Wild Young Hearts"""
13,"""Lonesome Crow"""
17,"""Graffiti Soul"""
26,"""Blues Six Pack"""



Shape: 658724 rows x 2 columns

Summary for numeric columns:
shape: (9, 3)
┌────────────┬─────────────┬────────────────────────────────┐
│ statistic  ┆ album_id    ┆ album                          │
│ ---        ┆ ---         ┆ ---                            │
│ str        ┆ f64         ┆ str                            │
╞════════════╪═════════════╪════════════════════════════════╡
│ count      ┆ 658724.0    ┆ 658724                         │
│ null_count ┆ 0.0         ┆ 0                              │
│ mean       ┆ 7.0209e6    ┆ null                           │
│ std        ┆ 5.5879e6    ┆ null                           │
│ min        ┆ 3.0         ┆ !                              │
│ 25%        ┆ 2.688685e6  ┆ null                           │
│ 50%        ┆ 5.557124e6  ┆ null                           │
│ 75%        ┆ 1.0878404e7 ┆ null                           │
│ max        ┆ 2.1461648e7 ┆ 🤘🏻 Rock Clásicos 80&90&2000 🤘🏻 │
└────────────┴─────────────┴────────────────────────────

In [15]:
# ---------- Clean albums_catalog from hidden duplicates ---------- #

albums_catalog = clean_and_filter_catalog(albums_catalog, 'album')

album_duplicates = (
    albums_catalog
        .group_by('album_clean')
        .agg([pl.len().alias('count')])
        .filter(pl.col('count') > 1)
        .sort('count')
)

print('Duplicated albums found:')
print(album_duplicates)

albums_dedup = (
    albums_catalog
        .group_by('album_clean')
        .agg(pl.min('album_id').alias('album_id_canonical'))
)

album_id_map = (
    albums_catalog
        .join(albums_dedup, on='album_clean', how='left')
        .select(['album_id', 'album_id_canonical'])
)

Duplicated albums found:
shape: (85_668, 2)
┌─────────────────────────────────┬───────┐
│ album_clean                     ┆ count │
│ ---                             ┆ ---   │
│ str                             ┆ u32   │
╞═════════════════════════════════╪═══════╡
│ Best Of Edm And Dubstep         ┆ 2     │
│ Party In Ibiza                  ┆ 2     │
│ Gencligin Gozyasları            ┆ 2     │
│ Real Deal Greatest Hits Volume… ┆ 2     │
│ Renkler Ve Islıklar             ┆ 2     │
│ …                               ┆ …     │
│ Gold                            ┆ 235   │
│ Collection                      ┆ 253   │
│ Best Of                         ┆ 502   │
│ Greatest Hits                   ┆ 706   │
│ Сборник                         ┆ 778   │
└─────────────────────────────────┴───────┘


In [16]:
# ---------- Clean Albums Catalog ----------

albums_catalog_clean = (
    albums_dedup
    .join(albums_catalog.select(['album_clean', 'album']), on='album_clean', how='left')
    .select([
        pl.col('album_id_canonical').alias('album_id'),
        pl.col('album_clean').alias('album')
    ])
    .unique()
)

In [17]:
# ---------- Dataframe artists_catalog set up ---------- #

artists_catalog = (
    catalog_names
        .filter(pl.col('type') == 'artist')
        .select([
            pl.col('id').alias('artist_id'),
            pl.col('name').alias('artist')
        ])
)

data_summary(artists_catalog, 'artists_catalog')


===== ARTISTS_CATALOG =====

Sample rows:


artist_id,artist
i64,str
4,"""Kenny Dorham"""
5,"""Max Roach"""
7,"""Francis Rossi"""
9,"""Status Quo"""
12,"""Phil Everly"""



Shape: 153581 rows x 2 columns

Summary for numeric columns:
shape: (9, 3)
┌────────────┬─────────────┬────────┐
│ statistic  ┆ artist_id   ┆ artist │
│ ---        ┆ ---         ┆ ---    │
│ str        ┆ f64         ┆ str    │
╞════════════╪═════════════╪════════╡
│ count      ┆ 153581.0    ┆ 153581 │
│ null_count ┆ 0.0         ┆ 0      │
│ mean       ┆ 3.9270e6    ┆ null   │
│ std        ┆ 3.5294e6    ┆ null   │
│ min        ┆ 4.0         ┆ 	Drake │
│ 25%        ┆ 678645.0    ┆ null   │
│ 50%        ┆ 3.297272e6  ┆ null   │
│ 75%        ┆ 6.143796e6  ┆ null   │
│ max        ┆ 1.6102782e7 ┆ 휘인   │
└────────────┴─────────────┴────────┘

Unique values (for each column):

Column: artist_id
shape: (153_581, 2)
┌───────────┬───────┐
│ artist_id ┆ count │
│ ---       ┆ ---   │
│ i64       ┆ u32   │
╞═══════════╪═══════╡
│ 7749967   ┆ 1     │
│ 677927    ┆ 1     │
│ 98533     ┆ 1     │
│ 1557532   ┆ 1     │
│ 2500058   ┆ 1     │
│ …         ┆ …     │
│ 359104    ┆ 1     │
│ 5681748   ┆ 1    

In [18]:
# ---------- Clean artists_catalog from hidden duplicates ---------- #

artists_catalog = clean_and_filter_catalog(artists_catalog, 'artist')

artist_duplicates = (
    artists_catalog
        .group_by('artist_clean')
        .agg([pl.len().alias('count')])
        .filter(pl.col('count') > 1)
        .sort('count')
)

print('Duplicated artists found:')
display(artist_duplicates)

artists_dedup = (
    artists_catalog
        .group_by('artist_clean')
        .agg(pl.min('artist_id').alias('artist_id_canonical'))
)

artist_id_map = (
    artists_catalog
        .join(artists_dedup, on='artist_clean', how='left')
        .select(['artist_id', 'artist_id_canonical'])
)

Duplicated artists found:


artist_clean,count
str,u32
"""Ab Liva""",2
"""Bt Project""",2
"""Терем Квартет""",2
"""Capo""",2
"""J Son""",2
…,…
"""Nero""",8
"""Nana""",8
"""Marina""",9


In [19]:
# ---------- Clean Artists Catalog ----------

artists_catalog_clean = (
    artists_dedup
    .join(artists_catalog.select(['artist_clean', 'artist']), on='artist_clean', how='left')
    .select([
        pl.col('artist_id_canonical').alias('artist_id'),
        pl.col('artist_clean').alias('artist')
    ])
    .unique()
)

In [20]:
# ---------- Dataframe genres_catalog set up ---------- #

genres_catalog = (
    catalog_names
        .filter(pl.col('type') == 'genre')
        .select([
            pl.col('id').alias('genre_id'),
            pl.col('name').alias('genre')
        ])
)

data_summary(genres_catalog, 'genres_catalog')


===== GENRES_CATALOG =====

Sample rows:


genre_id,genre
i64,str
0,"""all"""
1,"""eastern"""
2,"""rusrock"""
3,"""rusrap"""
4,"""postrock"""



Shape: 166 rows x 2 columns

Summary for numeric columns:
shape: (9, 3)
┌────────────┬────────────┬─────────┐
│ statistic  ┆ genre_id   ┆ genre   │
│ ---        ┆ ---        ┆ ---     │
│ str        ┆ f64        ┆ str     │
╞════════════╪════════════╪═════════╡
│ count      ┆ 166.0      ┆ 166     │
│ null_count ┆ 0.0        ┆ 0       │
│ mean       ┆ 206.228916 ┆ null    │
│ std        ┆ 317.553921 ┆ null    │
│ min        ┆ 0.0        ┆ african │
│ 25%        ┆ 41.0       ┆ null    │
│ 50%        ┆ 83.0       ┆ null    │
│ 75%        ┆ 127.0      ┆ null    │
│ max        ┆ 1542.0     ┆ work    │
└────────────┴────────────┴─────────┘

Unique values (for each column):

Column: genre_id
shape: (166, 2)
┌──────────┬───────┐
│ genre_id ┆ count │
│ ---      ┆ ---   │
│ i64      ┆ u32   │
╞══════════╪═══════╡
│ 383      ┆ 1     │
│ 74       ┆ 1     │
│ 64       ┆ 1     │
│ 119      ┆ 1     │
│ 45       ┆ 1     │
│ …        ┆ …     │
│ 2        ┆ 1     │
│ 1058     ┆ 1     │
│ 71       ┆ 1  

In [21]:
# ---------- Clean genres_catalog from hidden duplicates ---------- #

genres_catalog = clean_and_filter_catalog(genres_catalog, 'genre')

genre_duplicates = (
    genres_catalog
        .group_by('genre_clean')
        .agg([pl.len().alias('count')])
        .filter(pl.col('count') > 1)
        .sort('count')
)

print('Duplicated genres found:')
display(genre_duplicates)

# Deduplicate genre_catalog so that each genre has one unique id
genres_dedup = (
    genres_catalog
        .group_by('genre_clean')
        .agg(pl.min('genre_id').alias('genre_id_canonical'))
)

genre_id_map = (
    genres_catalog
        .join(genres_dedup, on='genre_clean', how='left')
        .select(['genre_id', 'genre_id_canonical'])
)

Duplicated genres found:


genre_clean,count
str,u32


In [22]:
# ---------- Clean Genres Catalog ----------

genres_catalog_clean = (
    genres_dedup
    .join(genres_catalog.select(['genre_clean', 'genre']), on='genre_clean', how='left')
    .select([
        pl.col('genre_id_canonical').alias('genre_id'),
        pl.col('genre_clean').alias('genre')
    ])
    .unique()
)

In [23]:
# ---------- Item dataset set up ---------- #

# Merge tracks_exploded with canonical ids and cleaned names
items = (
      tracks_exploded
            .join(artist_id_map.join(artists_dedup, on='artist_id_canonical'), 
                  left_on='artist_id', right_on='artist_id', how='left')
            .join(album_id_map.join(albums_dedup, on='album_id_canonical'),
                  left_on='album_id', right_on='album_id', how='left')
            .join(track_id_map.join(tracks_dedup, on='track_id_canonical'),
                  left_on='track_id', right_on='track_id', how='left')
            .join(genre_id_map.join(genres_dedup, on='genre_id_canonical'),
                  left_on='genre_id', right_on='genre_id', how='left')
)

rows, cols = items.shape
print(f'\nDataFrame shape: {rows} rows x {cols} columns')

# Keep only canonical ids and cleaned names, rename and drop full duplicates and missing values
items = (
    items
      .select([
          'track_id_canonical', 'track_clean',
          'artist_id_canonical', 'artist_clean',
          'album_id_canonical', 'album_clean',
          'genre_id_canonical', 'genre_clean'
      ])
      .rename({
          'track_id_canonical': 'track_id',
          'track_clean': 'track',
          'artist_id_canonical': 'artist_id',
          'artist_clean': 'artist',
          'album_id_canonical': 'album_id',
          'album_clean': 'album',
          'genre_id_canonical': 'genre_id',
          'genre_clean': 'genre'
      })
      .unique()                                 # Remove full duplicates
      .drop_nulls()                             # Drop rows with any missing values
)

data_summary(items, 'items')


DataFrame shape: 6765311 rows x 12 columns

===== ITEMS =====

Sample rows:


track_id,track,artist_id,artist,album_id,album,genre_id,genre
i64,str,i64,str,i64,str,i64,str
425568,"""4 Orchestral Suites Bwv1066 9 …",130279,"""Elaine Shaffer""",44267,"""Bach Works With Orchestra Menu…",23,"""Classical"""
34297053,"""Ahora Dice""",645282,"""J Balvin""",10812024,"""Latin Music Summer Party""",11,"""Pop"""
22415214,"""Bass Down Low 113 Bpm""",3398448,"""Running Trax""",3228674,"""50 Best Spinning Hits""",68,"""Electronics"""
24280,"""J S Bach Partita For Violin So…",12727,"""David Garrett""",10484661,"""Bach Eternal""",23,"""Classical"""
309822,"""Misirlou""",112044,"""Dick Dale His Del Tones""",6691999,"""Atelier Diego Hofner Vol 1""",11,"""Pop"""



Shape: 5486918 rows x 8 columns

Summary for numeric columns:
shape: (9, 9)
┌────────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬─────────┐
│ statistic  ┆ track_id  ┆ track     ┆ artist_id ┆ … ┆ album_id  ┆ album     ┆ genre_id  ┆ genre   │
│ ---        ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---     │
│ str        ┆ f64       ┆ str       ┆ f64       ┆   ┆ f64       ┆ str       ┆ f64       ┆ str     │
╞════════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═════════╡
│ count      ┆ 5.486918e ┆ 5486918   ┆ 5.486918e ┆ … ┆ 5.486918e ┆ 5486918   ┆ 5.486918e ┆ 5486918 │
│            ┆ 6         ┆           ┆ 6         ┆   ┆ 6         ┆           ┆ 6         ┆         │
│ null_count ┆ 0.0       ┆ 0         ┆ 0.0       ┆ … ┆ 0.0       ┆ 0         ┆ 0.0       ┆ 0       │
│ mean       ┆ 1.6127e7  ┆ null      ┆ 1.7980e6  ┆ … ┆ 5.2446e6  ┆ null      ┆ 79.32674  ┆ null    │
│ std        ┆

In [24]:
# Define a regex pattern for "safe" characters: letters, numbers, whitespace
safe_pattern = r'^[\w\s]+$'

# Filter rows where any column contains characters outside the safe pattern
problematic_rows = items.filter(
    ~pl.col('track').str.contains(safe_pattern) |
    ~pl.col('artist').str.contains(safe_pattern) |
    ~pl.col('album').str.contains(safe_pattern) |
    ~pl.col('genre').str.contains(safe_pattern)
)

print(f'Number of problematic rows: {problematic_rows.height}')

Number of problematic rows: 0


In [25]:
# ---------- Clean up interactions dataframe ---------- #

interactions = (
    interactions
        .with_columns(pl.col('started_at').dt.date().alias('started_at'))
        .drop('__index_level_0__')
)

# Drop observations with track_seq bigger than 95% percentile
track_seq_upper_threshold = (
    interactions
        .select(pl.col('track_seq').quantile(0.95))
        .item()
)

print('Upper threshold track_seq:', track_seq_upper_threshold)

interactions = (
    interactions
        .filter(pl.col('track_seq') <= track_seq_upper_threshold)
)

# Map track_id in interactions to canonical track_id
interactions = interactions.join(
    track_id_map,  # This should have 'track_id' -> 'track_id_canonical'
    on='track_id',
    how='left'
).with_columns([
    pl.col('track_id_canonical').alias('track_id')
]).select([
    'user_id', 'track_id', 'track_seq', 'started_at'
])

data_summary(interactions, 'interactions')

Upper threshold track_seq: 1837.0

===== INTERACTIONS =====

Sample rows:


user_id,track_id,track_seq,started_at
i32,i64,i16,date
0,99262,1,2022-07-17
0,589498,2,2022-07-19
0,590262,3,2022-07-21
0,590303,4,2022-07-22
0,590692,5,2022-07-22



Shape: 211506588 rows x 4 columns

Summary for numeric columns:
shape: (9, 5)
┌────────────┬───────────────┬──────────────┬──────────────┬────────────────────────────┐
│ statistic  ┆ user_id       ┆ track_id     ┆ track_seq    ┆ started_at                 │
│ ---        ┆ ---           ┆ ---          ┆ ---          ┆ ---                        │
│ str        ┆ f64           ┆ f64          ┆ f64          ┆ str                        │
╞════════════╪═══════════════╪══════════════╪══════════════╪════════════════════════════╡
│ count      ┆ 2.11506588e8  ┆ 2.11468901e8 ┆ 2.11506588e8 ┆ 211506588                  │
│ null_count ┆ 0.0           ┆ 37687.0      ┆ 0.0          ┆ 0                          │
│ mean       ┆ 687089.249788 ┆ 1.7834e7     ┆ 313.42529    ┆ 2022-08-28 05:51:33.513649 │
│ std        ┆ 396878.260481 ┆ 2.4075e7     ┆ 375.123715   ┆ null                       │
│ min        ┆ 0.0           ┆ 26.0         ┆ 1.0          ┆ 2022-01-01                 │
│ 25%        ┆ 342967

# Выводы

# === ЭТАП 2 ===

# EDA

Распределение количества прослушанных треков.

Наиболее популярные треки

In [26]:
tracks_by_listen_number = (
    interactions
        .group_by('track_id')
        .agg(pl.count('track_seq').alias('listen_count'))
        .join(tracks_catalog_clean.select(['track_id', 'track']), on='track_id', how='left')
        .sort('listen_count', descending=True)
)

top_tracks = tracks_by_listen_number.head(10)
display(top_tracks)

track_id,listen_count,track
i64,u32,str
40962,226193,"""In The End"""
37384,168681,"""Zombie"""
96513,160519,"""Seven Nation Army"""
17335,155411,"""Numb"""
587,154930,"""Smells Like Teen Spirit"""
416823,145742,"""Believer"""
78426489,141570,"""Rampampam"""
6264,136126,"""Sweet Dreams Are Made Of This"""
16703,135221,"""Monster"""


Наиболее популярные жанры

In [27]:
# Top 5 genres by listening number
genres_by_listen_count = (
    interactions
        .group_by('track_id')
        .agg(pl.len().alias('track_listen_count'))
        .join(
            items.select(['track_id', 'genre']).unique(['track_id', 'genre']),
            on='track_id',
            how='left'
        )
        .group_by('genre')
        .agg(pl.sum('track_listen_count').alias('listen_count'))
        .sort('listen_count', descending=True)
)

top_5_genres = genres_by_listen_count.head(5)
display(top_5_genres)

genre,listen_count
str,u32
"""Pop""",130468229
"""Rap""",101299398
"""Allrock""",98323160
"""Electronics""",85576146
"""Dance""",72818119


Треки, которые никто не прослушал

In [28]:
# Tracks that haven't been listened to by anybody
unlistened_tracks = (
    items
        .select(['track_id', 'track', 'artist', 'album', 'genre'])
        .unique('track_id')
        .join(
            interactions.select('track_id').unique(),
            on='track_id',
            how='anti'
        )
)

print(f'Number of unlistened tracks: {unlistened_tracks.height}')
print(unlistened_tracks.head(10))

Number of unlistened tracks: 22
shape: (10, 5)
┌──────────┬────────────────────────┬────────────────────────┬───────────────────────┬─────────────┐
│ track_id ┆ track                  ┆ artist                 ┆ album                 ┆ genre       │
│ ---      ┆ ---                    ┆ ---                    ┆ ---                   ┆ ---         │
│ i64      ┆ str                    ┆ str                    ┆ str                   ┆ str         │
╞══════════╪════════════════════════╪════════════════════════╪═══════════════════════╪═════════════╡
│ 70050701 ┆ Symphonic Suite Kiki S ┆ New Japan Philharmonic ┆ Symphonic Suite Kiki  ┆ Modern      │
│          ┆ Deliver…               ┆ World D…               ┆ S Deliver…            ┆             │
│ 77751634 ┆ Gypsy Drum Solo        ┆ Dio                    ┆ Holy Diver Live       ┆ Metal       │
│ 56166858 ┆ Панк Туристы           ┆ Скамеика Запасных      ┆ Новые Грехи Старые    ┆ Punk        │
│          ┆                        ┆       

# Преобразование данных

Преобразуем данные в формат, более пригодный для дальнейшего использования в расчётах рекомендаций.

# Сохранение данных

Сохраним данные в двух файлах в персональном S3-бакете по пути `recsys/data/`:
- `items.parquet` — все данные о музыкальных треках,
- `events.parquet` — все данные о взаимодействиях.

# Очистка памяти

Здесь, может понадобится очистка памяти для высвобождения ресурсов для выполнения кода ниже. 

Приведите соответствующие код, комментарии, например:
- код для удаление более ненужных переменных,
- комментарий, что следует перезапустить kernel, выполнить такие-то начальные секции и продолжить с этапа 3.

# === ЭТАП 3 ===

# Загрузка данных

Если необходимо, то загружаем items.parquet, events.parquet.

# Разбиение данных

Разбиваем данные на тренировочную, тестовую выборки.

# Топ популярных

Рассчитаем рекомендации как топ популярных.

# Персональные

Рассчитаем персональные рекомендации.

# Похожие

Рассчитаем похожие, они позже пригодятся для онлайн-рекомендаций.

# Построение признаков

Построим три признака, можно больше, для ранжирующей модели.

# Ранжирование рекомендаций

Построим ранжирующую модель, чтобы сделать рекомендации более точными. Отранжируем рекомендации.

# Оценка качества

Проверим оценку качества трёх типов рекомендаций: 

- топ популярных,
- персональных, полученных при помощи ALS,
- итоговых
  
по четырем метрикам: recall, precision, coverage, novelty.

# === Выводы, метрики ===

Основные выводы при работе над расчётом рекомендаций, рассчитанные метрики.